In [1]:
print(1)

1


# 🌙 Silhouette Score — Notes

## 1) Setup & Notation

- Dataset with \( n \) samples clustered into \(K\) clusters.
- For a sample \( i \), let \( C_i \) denote **the set of indices of points in the same cluster as \( i \)** (including \( i \) itself).
- \( d(i, j) \) is a chosen distance (e.g., Euclidean, cosine, etc.).

**Key idea:** Silhouette is **per-sample**. We compute a local score \( s(i) \) for every sample, then average.

---

## 2) Per-sample quantities

### Intra-cluster cohesion for sample \( i \)

$$
a(i) \;=\; \frac{1}{|C_i| - 1} \sum_{\substack{j \in C_i\\ j \ne i}} d(i, j)
$$

- This is the **mean distance from sample \( i \)** to **all other points in its own cluster**.
- **Not** the mean of *all pairwise* distances within the cluster; it is **per-point**.

### Nearest-other-cluster separation for sample \( i \)

For every other cluster \( C_k \neq C_i \), compute
$$
d(i, C_k) \;=\; \frac{1}{|C_k|} \sum_{j \in C_k} d(i, j),
$$
then take
$$
b(i) \;=\; \min_{C_k \ne C_i} \; d(i, C_k).
$$

- \( i \) is an **individual sample**, **not** the cluster center.
- \( b(i) \) is the **smallest average distance** from \( i \) to the points in another cluster.

### Silhouette coefficient for sample \( i \)

$$
s(i) \;=\; \frac{b(i) - a(i)}{\max\{a(i),\, b(i)\}} \;\in\; [-1, 1].
$$

**Interpretation:**
- \( s(i) \approx 1 \): \( i \) is well inside its own cluster (small \(a(i)\)) and far from others (large \(b(i)\)).
- \( s(i) \approx 0 \): \( i \) lies near a boundary ( \( a(i) \approx b(i) \) ).
- \( s(i) < 0 \): \( i \) may be closer to another cluster than to its own (potential misassignment).

---

## 3) Aggregation

- **Cluster-level silhouette**:
  $$
  S(C_k) \;=\; \frac{1}{|C_k|} \sum_{i \in C_k} s(i).
  $$
- **Overall silhouette score**:
  $$
  \bar{s} \;=\; \frac{1}{n} \sum_{i=1}^{n} s(i).
  $$

---

## 4) What Silhouette Measures

| Aspect | Quantity | Meaning |
|---|---|---|
| Cohesion | \( a(i) \) | How tightly \( i \) sits within its own cluster |
| Separation | \( b(i) \) | How far \( i \) is from the nearest **other** cluster |
| Balance | \( s(i) \) | Combines cohesion & separation for sample-level quality |

**Higher is better** (max \(=1\)); negative values flag likely misplacements.

---

## 5) Practical Guidance

- **Per-sample nature:** We **must** compute \( a(i) \) and \( b(i) \) for **every** sample, even within the same cluster; they differ point-to-point.
- **Distance metric matters:** The choice of \( d(\cdot,\cdot) \) (Euclidean, cosine, etc.) changes \( a(i), b(i) \), hence \( s(i) \).
- **Model selection:** Compare clusterings (e.g., different \( k \)) by the mean silhouette \( \bar{s} \).

---

## 6) Limitations & the “Adjacent Clusters” Question

Silhouette **rewards separation**:
$$
s(i) \text{ is high when } a(i)\text{ is small and } b(i)\text{ is large.}
$$

In realistic data, **clusters can be adjacent or touching**. Then for boundary points:
- \( a(i) \) is moderate/small (close to own cluster mates),
- \( b(i) \) is also moderate (near another cluster),
- hence \( b(i) - a(i) \approx 0 \Rightarrow s(i) \approx 0 \).

**Implications:**
- Moderate global scores (e.g., \(0.3\text{–}0.5\)) can still reflect *valid* clusterings when clusters naturally sit close together.
- Low silhouette doesn’t always mean “bad clustering” — it can mean **limited geometric separability** in the chosen distance space.

**Common failure modes:**
- **Overlapping or adjacent clusters:** Many \( s(i) \approx 0 \).
- **Elongated / non-globular clusters:** \( a(i) \) grows within the same cluster → silhouette penalizes.
- **Uneven densities:** Dense clusters get larger \( s \) than sparse ones.

**What to try:**
- Consider **alternative indices** (e.g., Calinski–Harabasz, Davies–Bouldin) that trade off compactness/separation differently.
- Try **other distance metrics** (cosine / correlation) or **transformations** (scaling, manifold learning) that increase separability.

---

## 7) Quick Numeric Illustration (Concept)

Two clusters \(C_1, C_2\) in 2D:

- For a point \( i \in C_1 \) near \(C_2\):
  - \( a(i) \) = average distance to points in \( C_1 \) (neighbors inside cluster).
  - \( b(i) \) = average distance to points in \( C_2 \); if clusters are adjacent, \( b(i) \) is not much larger than \( a(i) \).
  - Then \( s(i) \approx \frac{b(i)-a(i)}{\max(a(i),b(i))} \approx 0 \).

This reflects **geometric proximity**, not necessarily an incorrect label.

---

## 8) Summary

- **Silhouette is per-sample:** compute \( a(i), b(i), s(i) \) for **every** point.
- **High \( s(i) \)** ⇒ compact & well-separated around point \( i \).
- **Mean silhouette** \( \bar{s} \) summarizes global quality, but
  - can be **modest** when clusters are **adjacent**, **overlapping**, or **non-globular**.
- Use alongside other indices and domain knowledge; consider metric/feature transforms if separability is inherently low.


# 📊 Calinski–Harabasz Index (Variance Ratio Criterion)

## 1) Intuition

The **Calinski–Harabasz Index (CHI)** evaluates clustering quality by comparing how **tight** each cluster is internally and how **far apart** the clusters are globally.

Conceptually, it answers:

> “Do points within a cluster stay close to each other, and are different clusters far apart?”

So CHI rewards:
- **Low within-cluster dispersion** (compact clusters)
- **High between-cluster dispersion** (clusters well separated)

It behaves much like an **F-statistic** in ANOVA, comparing “explained” versus “unexplained” variance.

---

## 2) Notation

Let:
- \( n \): total number of samples
- \( k \): number of clusters
- \( x_i \): sample \( i \) (vector in feature space)
- \( C_j \): set of samples belonging to cluster \( j \)
- \( c_j \): centroid (mean vector) of cluster \( j \)
- \( \bar{x} \): global mean of all samples

---

## 3) Between-cluster dispersion (how far cluster centers are from the global mean)

$$
B_k = \sum_{j=1}^{k} |C_j| \, \| c_j - \bar{x} \|^2
$$

**Interpretation:**
- Measures how much each cluster’s centroid deviates from the overall mean.
- Weighted by the cluster’s size \( |C_j| \).
- Large \(B_k\) means centroids are far apart → **good cluster separation**.

---

## 4) Within-cluster dispersion (how tight each cluster is)

$$
W_k = \sum_{j=1}^{k} \sum_{x_i \in C_j} \| x_i - c_j \|^2
$$

**Interpretation:**
- Measures how far points are from their own cluster centroid.
- Low \(W_k\) means clusters are internally compact → **good cohesion**.

---

## 5) The Calinski–Harabasz Index

Combine both as a ratio:

$$
CH(k) = \frac{B_k / (k - 1)}{W_k / (n - k)}
$$

**Interpretation:**
- The numerator: “average between-cluster variance”
- The denominator: “average within-cluster variance”
- Higher ratio → clusters explain more of the total variance in the data.

---

## 6) Analogy with ANOVA (F-statistic)

The formula mirrors an ANOVA \( F \)-ratio:

$$
F = \frac{\text{Between-group variance} / \text{df}_1}{\text{Within-group variance} / \text{df}_2}
$$

Here:
- \( \text{df}_1 = k - 1 \)
- \( \text{df}_2 = n - k \)

Like in ANOVA:
- The **between** term captures variability explained by cluster means.
- The **within** term captures residual variability inside each cluster.

---

## 7) Interpretation Summary

| Scenario | Behavior | CHI Outcome |
|-----------|-----------|-------------|
| Clusters far apart and tight inside | \( B_k ↑, W_k ↓ \) | High CHI |
| Clusters overlapping or diffuse | \( B_k ↓, W_k ↑ \) | Low CHI |
| Too few clusters | \( B_k \) small (centroids close) | Low CHI |
| Too many clusters | \( W_k \) small but \( B_k \) barely improves | CHI plateaus or drops |

Thus, **the higher the CHI, the better**.
The optimal number of clusters \( k^* \) is usually where \( CH(k) \) peaks.

---

## 8) Properties

| Property | Description |
|-----------|-------------|
| Range | \( (0, \infty) \) |
| Direction | Higher = better clustering |
| Sensitivity | Sensitive to \(k\) and to uneven cluster sizes |
| Best for | Convex, roughly spherical clusters (e.g. K-Means) |
| Computational cost | Low (depends only on centroids and distances) |

---

## 9) Limitations

- Assumes **convex, isotropic clusters** → works best with K-Means-type partitions.
- Can **favor higher \(k\)** since more clusters reduce within-cluster variance.
- Not robust to **noise or highly imbalanced cluster sizes**.

Use it primarily for:
- Comparing results from the same clustering algorithm across different \(k\).
- Quick evaluation when you expect roughly spherical, well-separated clusters.

---

## 10) Summary Table

| Component | Formula | Interpretation |
|------------|----------|----------------|
| Between-cluster variance | \( B_k = \sum_j |C_j| \| c_j - \bar{x} \|^2 \) | Cluster centroids far from global mean |
| Within-cluster variance | \( W_k = \sum_j \sum_{x_i \in C_j} \| x_i - c_j \|^2 \) | Points close to their own centroid |
| Index | \( CH(k) = \frac{B_k / (k - 1)}{W_k / (n - k)} \) | Ratio of separation to compactness |
| Higher CHI | Clusters dense and far apart | Indicates better clustering |

---

✅ **In short:**
The Calinski–Harabasz Index quantifies *how much of the total variance in the data is explained by the clustering*.
Large \(CH(k)\) → high separation between centroids and low spread within clusters → **strong, well-structured clustering**.


# 🔷 Davies–Bouldin Index (DBI)

## 1) Intuition

The **Davies–Bouldin Index (DBI)** measures the average “similarity” between each cluster and its most similar neighbor.
It captures both **compactness** (how tight a cluster is) and **separation** (how far clusters are from each other).

Conceptually:

> A good clustering has compact clusters that are far apart —
> DBI rewards that by giving lower scores to better structures.

---

## 2) Notation

Let:
- \( n \): total number of samples
- \( k \): number of clusters
- \( C_i \): the set of samples in cluster \(i\)
- \( c_i \): centroid (mean vector) of cluster \(i\)
- \( d(x, y) \): distance measure (e.g. Euclidean)

---

## 3) Cluster scatter — how tight a cluster is

For each cluster \( C_i \):

$$
s_i = \frac{1}{|C_i|} \sum_{x \in C_i} \| x - c_i \|
$$

- \( s_i \) = **average distance** from points in cluster \(i\) to their centroid.
- Low \( s_i \) → cluster points are compact and cohesive.
- High \( s_i \) → cluster is wide or has outliers.

---

## 4) Inter-cluster separation — how far clusters are apart

For any two clusters \( C_i \) and \( C_j \):

$$
d_{ij} = \| c_i - c_j \|
$$

- \( d_{ij} \) measures the **distance between the centroids** of clusters \(i\) and \(j\).
- Large \( d_{ij} \) → clusters are well-separated.
- Small \( d_{ij} \) → clusters are close or overlapping.

---

## 5) Pairwise cluster similarity

For each pair of clusters \( (i, j) \), define:

$$
R_{ij} = \frac{s_i + s_j}{d_{ij}}
$$

- \( s_i + s_j \): combined spread (how wide both clusters are).
- \( d_{ij} \): distance between their centers.
- Larger \( R_{ij} \) → clusters are more similar (closer or more diffuse).
- Smaller \( R_{ij} \) → clusters are distinct and tight.

---

## 6) Cluster-level “worst-case” similarity

For each cluster \( i \), find the most similar (most overlapping) neighbor:

$$
R_i = \max_{j \ne i} R_{ij}
$$

- \( R_i \): worst-case ratio for cluster \(i\).
- Large \( R_i \) → cluster \(i\) has a neighbor too close or too spread out.
- Small \( R_i \) → cluster \(i\) is compact and well-separated.

---

## 7) The Davies–Bouldin Index

Finally, average over all clusters:

$$
DBI = \frac{1}{k} \sum_{i=1}^{k} R_i
$$

This gives the **mean of all clusters’ worst-case similarity scores**.

**Lower DBI = better clustering**
→ clusters are compact and well-separated.

---

## 8) Interpretation

| Range | Meaning |
|--------|----------|
| **Low (≈ 0–0.5)** | Strong, compact, clearly separated clusters |
| **Moderate (≈ 0.5–1.0)** | Some overlap or adjacent clusters |
| **High (> 1.0)** | Overlapping or diffuse clusters |

**Goal:** Minimize \( DBI \)

---

## 9) Comparison with Other Metrics

| Metric | Optimize | Focus | Nature |
|---------|-----------|--------|--------|
| **Silhouette Score** | Maximize | Cohesion & separation per point | Point-level |
| **Calinski–Harabasz** | Maximize | Global variance ratio | Cluster-level (variance-based) |
| **Davies–Bouldin** | Minimize | Pairwise cluster overlap | Cluster-level (distance-based) |

DBI focuses on **worst-case cluster similarity**, so it’s sensitive to overlapping or uneven clusters.

---

## 10) Properties

| Property | Description |
|-----------|--------------|
| Range | \( [0, \infty) \) |
| Direction | Lower = better |
| Sensitivity | Sensitive to overlapping or unbalanced clusters |
| Complexity | \( O(k^2) \) (pairwise centroid comparisons) |
| Best for | Convex, centroid-based clustering (e.g. K-Means) |

---

## 11) Limitations

- **Assumes spherical clusters:** works best for centroid-based methods.
- **Sensitive to outliers:** large \( s_i \) can distort results.
- **Can penalize valid adjacent clusters:** if centroids are close, \( d_{ij} \) small → larger DBI.

---

## 12) Summary Table

| Concept | Formula | Meaning |
|----------|----------|----------|
| Cluster scatter | \( s_i = \frac{1}{|C_i|} \sum_{x \in C_i} \|x - c_i\| \) | Cluster’s average radius |
| Cluster separation | \( d_{ij} = \|c_i - c_j\| \) | Distance between centroids |
| Pairwise similarity | \( R_{ij} = \frac{s_i + s_j}{d_{ij}} \) | Overlap between two clusters |
| Worst similarity per cluster | \( R_i = \max_{j \ne i} R_{ij} \) | Closest or most overlapping neighbor |
| Index | \( DBI = \frac{1}{k} \sum_i R_i \) | Average worst-case cluster overlap |
| Optimization | **Minimize** \( DBI \) | Lower = better |

---

✅ **In short:**
The **Davies–Bouldin Index** evaluates cluster quality by averaging how similar each cluster is to its most similar neighbor.
It rewards clustering with **compact clusters** (low \( s_i \)) and **large inter-cluster distances** (high \( d_{ij} \)).
A **smaller DBI** means the clusters are tight, distinct, and well-separated.


# 🧩 Contingency Matrix & Pair Confusion Matrix — Notes

## 1) Contingency Matrix

- A **contingency matrix** compares *true labels* and *predicted clusters*.
- Each element \( n_{ij} \) counts samples belonging to true class \(i\) and assigned to predicted cluster \(j\):

$$
n_{ij} = |\{ x : y_{\text{true}}(x) = i,\, y_{\text{pred}}(x) = j \}|
$$

| True ↓ / Predicted → | Cluster 1 | Cluster 2 | Cluster 3 |
|----------------------|-----------|-----------|-----------|
| Class A | 8 | 2 | 0 |
| Class B | 1 | 6 | 3 |
| Class C | 0 | 2 | 7 |

- Used as the **base for most external clustering metrics**, e.g.:
  - **Mutual Information**, **Normalized MI**
  - **Homogeneity / Completeness / V-measure**
  - **Rand Index / Adjusted Rand Index**

---

## 2) Pair Confusion Matrix

- Reformulates clustering comparison in terms of **pairs of samples**.
- Each sample pair \((x_a, x_b)\) falls into one of 4 cases:

| Case | Meaning | Condition |
|------|----------|-----------|
| **TP** | True Positive | same true label **and** same predicted cluster |
| **FP** | False Positive | different true labels but **same predicted cluster** |
| **FN** | False Negative | same true label but **different predicted clusters** |
| **TN** | True Negative | different true labels **and** different predicted clusters |

| | Predicted same | Predicted different |
|--|----------------|--------------------|
| True same | TP | FN |
| True different | FP | TN |

---

## 3) Connections to Metrics

| Metric | Derived from | Measures | Optimized |
|---------|---------------|-----------|-------------|
| **Rand Index** | Pair Confusion Matrix | Fraction of agreeing pairs: \( (TP + TN) / \text{total} \) | ↑ Maximize |
| **Adjusted Rand Index** | Pair Confusion Matrix | Rand Index corrected for chance | ↑ Maximize |
| **Mutual Information / NMI** | Contingency Matrix | Shared information between labels | ↑ Maximize |
| **Homogeneity / Completeness / V-measure** | Contingency Matrix (via entropy & MI) | Purity and completeness of cluster labeling | ↑ Maximize |

---

✅ **In short:**
- The **Contingency Matrix** counts overlaps between true classes and predicted clusters.
- The **Pair Confusion Matrix** counts *pairwise agreements/disagreements* between them.
All major **supervised clustering metrics** — Rand, ARI, MI, and V-measure — are derived from one of these two foundations.
